In [1]:
from pyspark.sql import SparkSession

In [47]:
file = "data/test-graph.txt"
max_iter = 100

In [18]:
# Load the data
textFile = sc.textFile(file)
textFile.count()

16

In [19]:
dataFile = textFile.filter(lambda l: len(l) > 0 and l[0] != '#')
dataFile.count()

11

In [36]:
def to_tuple(line, delim='\t'):
    strings = line.split(delim)[:2]
    return (strings[0], strings[1])
    
edges = dataFile.map(to_tuple)
edges.setName('Edges')
assert edges.filter(lambda t: len(t) == 2).count() == dataFile.count()
type(edges)

pyspark.rdd.PipelinedRDD

In [44]:
def switch_key_value(kv):
    return (kv[1], kv[0])

def compose(r1, r2):
    """ Compose 2 relations represented by PairRDDs. """
    r1_flip = r1.map(switch_key_value)
    joined = r1_flip.join(r2)
    return joined.values()

paths = edges
for i in range(4):
    print("###########")
    paths = compose(paths, edges)
    for p in paths.collect():
        print(p)

###########
('3', '5')
('9', '6')
('2', '4')
('5', '7')
('5', '8')
('5', '9')
('10', '7')
('10', '8')
('10', '9')
('6', '10')
('0', '2')
('4', '6')
('1', '3')
###########
('2', '5')
('1', '4')
('0', '3')
('6', '6')
('5', '10')
('10', '10')
('3', '6')
('9', '7')
('9', '8')
('9', '9')
('4', '7')
('4', '8')
('4', '9')
###########
('1', '5')
('5', '6')
('10', '6')
('9', '10')
('4', '10')
('2', '6')
('0', '4')
('6', '7')
('6', '8')
('6', '9')
('3', '7')
('3', '8')
('3', '9')
###########
('9', '6')
('4', '6')
('5', '7')
('5', '8')
('5', '9')
('10', '7')
('10', '8')
('10', '9')
('2', '7')
('2', '8')
('2', '9')
('0', '5')
('6', '10')
('3', '10')
('1', '6')


In [48]:
paths = edges
all_paths = edges
for i in range(max_iter):
    print("###########")
    paths = compose(paths, edges)
    # Leave only new paths
    paths = paths.subtract(all_paths)
    # Finish, when no more paths added
    if paths.isEmpty():
        break
    
    # Add new paths to all paths
    all_paths = all_paths.union(paths)
    
    for p in paths.collect():
        print(p)

        
print("$$$$$$$$$$$$$ ALL PATHS $$$$$$$$$$$$$$")
for p in all_paths.collect():
        print(p)

###########
('5', '7')
('10', '9')
('9', '6')
('2', '4')
('1', '3')
('5', '8')
('0', '2')
('5', '9')
('10', '7')
('6', '10')
('10', '8')
('3', '5')
('4', '6')
###########
('5', '10')
('4', '7')
('6', '6')
('9', '8')
('9', '9')
('9', '7')
('2', '5')
('1', '4')
('10', '10')
('0', '3')
('4', '9')
('3', '6')
('4', '8')
###########
('4', '10')
('0', '4')
('3', '8')
('3', '7')
('1', '5')
('2', '6')
('3', '9')
###########
('2', '9')
('1', '6')
('0', '5')
('2', '8')
('3', '10')
('2', '7')
###########
('1', '8')
('2', '10')
('1', '9')
('1', '7')
('0', '6')
###########
('1', '10')
('0', '8')
('0', '9')
('0', '7')
###########
('0', '10')
###########
$$$$$$$$$$$$$ ALL PATHS $$$$$$$$$$$$$$
('0', '1')
('1', '2')
('2', '3')
('3', '4')
('4', '5')
('5', '6')
('6', '7')
('6', '8')
('6', '9')
('9', '10')
('10', '6')
('5', '7')
('10', '9')
('9', '6')
('2', '4')
('1', '3')
('5', '8')
('0', '2')
('5', '9')
('10', '7')
('6', '10')
('10', '8')
('3', '5')
('4', '6')
('5', '10')
('4', '7')
('6', '6')
('9', '8')